In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import deque
import random

2026-01-09 12:14:44.306644: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class MiniPPO:
    def __init__(self):
        self.device='cuda' if torch.cuda.is_available() else 'cpu'
        self.model=GPT2LMHeadModel.from_pretrained('gpt2').to(self.device)
        self.tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token=self.tokenizer.eos_token
        self.optimizer=optim.Adam(self.model.parameters(), lr=1e-5)
        self.epsilon=0.2